In [1]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1:Rajieev K
- Name 2:Manjula K
- Name 3:Araya M

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [2]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

#### Answer:
 * There are 11,314 instances.
 * Each instance represents news articles.
 * There are 20 classes and each class represents a topic.
 * No not for all cases. Obscure phrases might hold true but more time sensitive classifications related to topics that change often over time like computer graphics will not be generalized by using a dataset generated at a different timeframe. For this we can refer [here](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html).
 * It is a sparse matrix.





___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [3]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
#p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:',euclidean(a,c) , 'bc:',euclidean(b,c))
print('Cosine Distance\n ab:',cosine(a,b) , 'ac:', cosine(a,c), 'bc:', cosine(b,c))

print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:',jaccard(a>0, b>0) , 'ac:', jaccard(a>0, c>0), 'bc:', jaccard(b>0, c>0))
print('\n\nThe most appropriate distance is Euclidean and Cosine')
#print(p)

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398236 bc: 0.9177794226661624
Cosine Distance
 ab: 0.603371411376 ac: 0.707027614956 bc: 0.421159534335
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.882113821138 ac: 0.875471698113 bc: 0.908794788274


The most appropriate distance is Euclidean and Cosine


### Answer
For this data set, Euclidean and Cosine are better to use. Because we expect articles on autos and motorcycles to be close but artciles on graphics to be far. And, Euclidean and cosine distances follow this expectation. Euclidean distance between Recreational autos and recreational motorcycles (b&c) is 0.917 (smaller than others) which looks logical. Also, Cosine distance between Recreational autos and recreational motorcycles (b&c) is 0.421 (smaller than others) which seems logical. Jaccard dissimilarity doesn't tell much about these 3 articles, so it is not appropriate here. For definition of these similarities we can get info in [similarty](http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/).  

___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [4]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 
from sklearn import metrics

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

    
# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
#K=5
K = [1,2,3,4,5]

for k in K:
 clf = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric='euclidean')
 clf.fit(X_train,y_train) 
 y_pred=clf.predict(X_test)
 acc=metrics.accuracy_score(y_test, y_pred)
 print('Accuracy of classifier with %d neighbors is: %.2f'%(k,acc))

print("k=1 gives the best accuracy as seen above")



//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Accuracy of classifier with 1 neighbors is: 0.60
Accuracy of classifier with 2 neighbors is: 0.57
Accuracy of classifier with 3 neighbors is: 0.55
Accuracy of classifier with 4 neighbors is: 0.53
Accuracy of classifier with 5 neighbors is: 0.52
k=1 gives the best accuracy as seen above


The best accuracy is 0.60 with 1 neighbor.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?


KD tree is not better for sparse data. In KD tree decisions or splits are made based on values, thus when the data is sparse containing many zeros the tree will not be split efficient, but instead the tree will be end up being severely unbalanced and deep. Which in turn makes KD tree  computationally inefficient.


___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [5]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)
# For Euclidean Metric
clf = NearestCentroid(metric='euclidean')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with euclidean metric is: %.2f'%(acc))

# For Cosine Metric
clf = NearestCentroid(metric='cosine')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with cosine metric is: %.2f'%(acc))

# For Manhattan Metric
clf = NearestCentroid(metric='manhattan')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with manhattan metric is: %.2f'%(acc))

p='cosine'
print('The best distance metric is: ', p)

Accuracy of classifier with euclidean metric is: 0.41


//anaconda/lib/python3.5/site-packages/sklearn/neighbors/nearest_centroid.py:138: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


Accuracy of classifier with cosine metric is: 0.49
Accuracy of classifier with manhattan metric is: 0.18
The best distance metric is:  cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Total different conditional probabilities will be total number of attributes, which happens to be 130107. Total priors will be equal to total number of class, which is 20.


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

For continuous variables Gaussian naive is the best. Because, we can use discritization and distribution modelling of continuous features with Naive Bayes. Gaussian Multinomial is for sparse matrix such as text classification. Whereas Gaussian Bernoulli is for boolean data.  
If the data is sparse then Multinomial Naive Bayes is the best.


___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


# Alpha 1.0 And Binarize = 0.0
clf_mnb = MultinomialNB(alpha=1.0)
clf_bnb = BernoulliNB(alpha=1.0, binarize=0.0)

# For Multinomial NB
clf_mnb.fit(X_train,y_train)
y_pred=clf_mnb.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with alpha=1.0 and MultinomialNB is: %.2f'%(acc))

# For Bernoulli NB
clf_bnb.fit(X_train,y_train)
y_pred=clf_bnb.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with alpha=1.0 ,binarize=0.0 and BernoulliNB is: %.2f'%(acc))

# Alpha 0.0 And Binarize = 0.0
clf_mnb = MultinomialNB(alpha=0)
clf_bnb = BernoulliNB(alpha=0, binarize=0.0)

# For Multinomial NB
clf_mnb.fit(X_train,y_train)
y_pred=clf_mnb.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with alpha=0.0 and MultinomialNB is: %.2f'%(acc))

# For Bernoulli NB
clf_bnb.fit(X_train,y_train)
y_pred=clf_bnb.predict(X_test)
acc=metrics.accuracy_score(y_test, y_pred)
print('Accuracy of classifier with alpha=0.0 ,binarize=0.0 and BernoulliNB is: %.2f'%(acc))

            
print('These classifiers are so fast because training is done by evaluating a closed-form expressions rather than iterative approximation' , p)
print('The alpha values control smoothing parameter, 0 being no smoothing. If a class and feature value doesnot occur in the test data, it nullifies other probabilities  in the model', p)


Accuracy of classifier with alpha=1.0 and MultinomialNB is: 0.71
Accuracy of classifier with alpha=1.0 ,binarize=0.0 and BernoulliNB is: 0.62


//anaconda/lib/python3.5/site-packages/sklearn/naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


Accuracy of classifier with alpha=0.0 and MultinomialNB is: 0.11


//anaconda/lib/python3.5/site-packages/sklearn/naive_bayes.py:801: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -
//anaconda/lib/python3.5/site-packages/sklearn/naive_bayes.py:820: RuntimeWarning: divide by zero encountered in log
  neg_prob = np.log(1 - np.exp(self.feature_log_prob_))


Accuracy of classifier with alpha=0.0 ,binarize=0.0 and BernoulliNB is: 0.04
These classifiers are so fast because training is done by evaluating a closed-form expressions rather than iterative approximation cosine
The alpha values control smoothing parameter, 0 being no smoothing. If a class and feature value doesnot occur in the test data, it nullifies other probabilities  in the model cosine


//anaconda/lib/python3.5/site-packages/sklearn/naive_bayes.py:823: RuntimeWarning: invalid value encountered in add
  jll += self.class_log_prior_ + neg_prob.sum(axis=1)


________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.